In [1]:
import numpy as np
import numpy.linalg as la
import scipy.optimize as scop
import os
from os import listdir
from os.path import isfile, join

In [2]:
def newton(A, B, C, D, n):
    E = np.matmul(A, B) + np.matmul(C, D)
    zrs = np.zeros((n,n))
    k = 1 / (1 + la.norm(B, 1) * la.norm(A, np.inf) + la.norm(C, 1) * la.norm(D, np.inf))  
    print("coefficient = ", k)
    equation = lambda X: k * (np.matmul(np.matmul(A, X), B) + np.matmul(np.matmul(C, X), D) + E) - X
    sol = scop.newton_krylov(equation, zrs, verbose = True, f_tol = pow(10, - 13), maxiter = n * n // 4)
    %timeit -r 1 -n 1 scop.newton_krylov(equation, zrs, verbose = True, f_tol = pow(10, - 13), maxiter = n * n // 4)
    return ((sol != 0).astype(int))

In [3]:
def parse_graph(_file):
    nc = 0
    with open(_file, 'r') as graph:
        for line in graph:
            terms = line.split()
            nc = max(nc, int(terms[0]), int(terms[2]))
    nc += 1
    
    A = np.zeros((nc,nc))
    B = np.zeros((nc,nc))
    C = np.zeros((nc,nc))
    D = np.zeros((nc,nc))
    with open(_file, 'rt') as graph:
        for line in graph:
            terms = line.split()
            if terms[1] == "SCOR":
                A[int(terms[0])][int(terms[2])] = 1
            elif terms[1] == "SCO":
                B[int(terms[0])][int(terms[2])] = 1
            elif terms[1] == "TR":
                C[int(terms[0])][int(terms[2])] = 1
            elif terms[1] == "T":
                D[int(terms[0])][int(terms[2])] = 1
                
    print("As = ", np.sum(A), "Bs = ", np.sum(B), "Cs = ", np.sum(C), "Ds = ", np.sum(D))
    return A, B, C, D, nc

In [4]:
def main(graphfile, resultfile):
    A, B, C, D, n = parse_graph(graphfile)
    print("verteces_count: ", n)

    res = newton(A, B, C, D, n)
    resf = open(resultfile, 'w')
    count = 0
    for i in range(n):
        for j in range(n):
            if res[i][j] == 1: 
                count += 1
            resf.write("%d " %res[i][j])
        resf.write("\r\n")
    resf.close()
    print("count = ", count)

In [5]:
gr_path = "/home/ju-sus/Documents/CFPQ/data/RDF-graphs"
res_path = "/home/ju-sus/Documents/CFPQ/results"
graphs = [f for f in listdir(gr_path) if isfile(join(gr_path, f))]
print(graphs)
results = ["res-" + f for f in graphs]
print(results)

['people_pets.txt', 'atom-primitive.txt', 'skos.txt', 'core.txt', 'generations.txt', 'biomedical-mesure-primitive.txt', 'pizza.txt', 'univ-bench.txt', 'travel.txt', 'foaf.txt', 'wine.txt', 'funding.txt']
['res-people_pets.txt', 'res-atom-primitive.txt', 'res-skos.txt', 'res-core.txt', 'res-generations.txt', 'res-biomedical-mesure-primitive.txt', 'res-pizza.txt', 'res-univ-bench.txt', 'res-travel.txt', 'res-foaf.txt', 'res-wine.txt', 'res-funding.txt']


In [6]:
for i in range(len(graphs)):
    print()
    print(graphs[i])
    graph = os.path.join(gr_path, graphs[i])
    result = os.path.join(res_path, results[i])
    %timeit -r 1 -n 1 main(graph, result)


people_pets.txt
As =  33.0 Bs =  33.0 Cs =  161.0 Ds =  161.0
verteces_count:  337
coefficient =  0.0001290489095367144
0:  |F(x)| = 6.16783e-12; step 1
1:  |F(x)| = 5.42101e-20; step 1
0:  |F(x)| = 6.16783e-12; step 1
1:  |F(x)| = 5.42101e-20; step 1
324 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
count =  9472
828 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

atom-primitive.txt
As =  122.0 Bs =  122.0 Cs =  138.0 Ds =  138.0
verteces_count:  291
coefficient =  6.502796202367018e-05
0:  |F(x)| = 2.12818e-09; step 1
1:  |F(x)| = 2.43945e-19; step 1
0:  |F(x)| = 2.12818e-09; step 1
1:  |F(x)| = 2.43945e-19; step 1
110 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
count =  15454
368 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

skos.txt
As =  1.0 Bs =  1.0 Cs =  70.0 Ds =  70.0
verteces_count:  144
coefficient =  0.001272264631043257
0:  |F(x)| = 1.6156e-06; step 1
1:  |F(x)| = 2.08167e-17; step 1
0:  |F(x)| = 1.6156e-06; step 